# Lens Iso
###### (http://www.philipzucker.com/lens-as-a-divisibility-relation-goofin-off-with-the-algebra-of-types/)
We can think of types in terms of their cardinalities, meaning polynomials using natural numbers. Types that contain the same number of inhibitants are isomorphic. Therefore, types modulo isomorphisms are equivalent to the natural numbers.

In [9]:
:ext TypeOperators

import qualified Data.Void as V

type a .+ b = Either a b
type a .* b = (a, b)
type b .^ a = a -> b

type O = V.Void
type I = ()
type Succ a = I .+ a 
type Two = Succ I
type Three = Succ Two
type Four = Succ Three

Polymorphic types are like polynomials where instantiating the type is like solving for a value. The lens library has the `Iso' s a` type which represents all the isomorphisms between some `a` and some `s`. This is the monomorphic version of `Iso s t a b` which is defined as:

In [5]:
:ext MultiParamTypeClasses
:ext RankNTypes
:m Data.Functor.Identity
:m Data.Functor.Const

class Profunctor p where
  dimap :: (s -> a) -> (b -> t) -> p a b -> p s t

instance Profunctor (->) where
  dimap sa bt ab = bt . ab . sa
  
type Iso s t a b = forall f p. (Functor f, Profunctor p) => p a (f b) -> p s (f t)
type Iso' s a = Iso s s a a

-- create an iso from the two morphisms. Note that this is (s -> a) and (a -> s) in the monomorphic case.
iso :: (s -> a) -> (b -> t) -> Iso s t a b
iso sa bt = dimap sa (fmap bt)

unIso :: Iso s t a b -> (s -> a, b -> t)
unIso pafb'psft = (sa, bt) where
  sa s = getConst $ pafb'psft Const s
  bt b = runIdentity $ pafb'psft (const $ Identity b) undefined

from :: Iso s t a b -> Iso b a t s
from pafb'psft ptfs = dimap bt (fmap sa) ptfs where
  (sa, bt) = unIso pafb'psft
  
mbListIso :: Iso' [a] (Maybe a)
mbListIso = iso lToMb mbToL where
  lToMb [] = Nothing
  lToMb (x:_) = Just x
  mbToL Nothing = []
  mbToL (Just x) = [x]
  
runIso :: Iso s t a b -> s -> a
runIso iso = getConst . iso Const
  
runIso mbListIso [()]
runIso (from mbListIso) (Just ())

Just ()

[()]

Now we can start writing some proofs

In [6]:
type a ~~ b = Iso' a b

refl :: a ~~ a
refl = id

compose :: (a ~~ b) -> (b ~~ c) -> (a ~~ c)
compose = (.)

rev :: a ~~ b -> b ~~ a
rev = from

proof1 :: (I .+ I) ~~ Two
proof1 = id

Writes proofs for the associativity property of addition and multiplication.

In [7]:
plusAssoc :: (a .+ (b .+ c)) ~~ ((a .+ b) .+ c)
plusAssoc = iso assoc unAssoc where
  assoc (Left a) = Left $ Left a
  assoc (Right (Left b)) = Left $ Right b
  assoc (Right (Right c)) = Right c
  unAssoc (Right c) = Right $ Right c
  unAssoc (Left (Right b)) = Right $ Left b
  unAssoc (Left (Left a)) = Left a
  
mulAssoc :: (a .* (b .* c)) ~~ ((a .* b) .* c)
mulAssoc = iso assoc unAssoc where
  assoc (a, (b, c)) = ((a, b), c)
  unAssoc ((a, b), c) = (a, (b, c))

and the identity property

In [11]:
plusId :: (a .+ O) ~~ a
plusId = iso to fro where
  to (Left a) = a
  to (Right x) = V.absurd x
  fro a = Left a
  
mulId :: (a .* I) ~~ a
mulId = iso to fro where
  to (a, _) = a
  fro a = (a, ())

commutivity property

In [12]:
plusComm :: (a .+ b) ~~ (b .+ a)
plusComm = iso to to where
  to (Left a) = Right a
  to (Right b) = Left b
  
mulComm :: (a .* b) ~~ (b .* a)
mulComm = iso to to where
  to (a, b) = (b, a)
  to (b, a) = (a, b)

distributive property

In [14]:
distr :: (a .* (b .+ c)) ~~ ((a .* b) .+ (a.* c))
distr = iso to fro where
  to (a, Left b) = Left (a, b)
  to (a, Right c) = Right (a, c)
  fro (Left (a, b)) = (a, Left b)
  fro (Right (a, c)) = (a, Right c)
  
mult0 :: (a .* O) ~~ O
mult0 = iso to fro where
  to (a, x) = x
  fro x = (V.absurd x, x)

We have combinators for lifting `Iso`s into Bifunctors

In [23]:
lefting :: (a ~~ b) -> ((a .+ c) ~~ (b .+ c))
lefting a'b = iso to fro where
  (ab, ba) = unIso a'b
  to (Left a) = Left $ ab a 
  to (Right c) = Right c
  fro (Left b) = Left $ ba b
  fro (Right c) = Right c
  
righting :: (a ~~ b) -> ((c .+ a) ~~ (c .+ b))
righting a'b = iso to fro where
  (ab, ba) = unIso a'b
  to (Left c) = Left c
  to (Right a) = Right $ ab a
  fro (Left c) = Left c
  fro (Right b) = Right $ ba b

Here's a more complicated proof now available to us

In [31]:
twoTwoFour :: (Two .+ Two) ~~ Four
twoTwoFour = rev plusAssoc

-- plusAssoc is
-- plusAssoc :: (a .+ (b .+ c)) ~~ ((a .+ b) .+ c)
-- by reversing it we get
-- ((a .+ b) .+ c) ~~ (a .+ (b .+ c))

-- Two is
-- I .+ I
-- Four is
-- I .+ (I .+ (I .+ I))

this :: (I .+ I .+ (I .+ I)) ~~ (I .+ (I .+ (I .+ I)))
this = rev plusAssoc

-- So `a` is I, `b` is I, `c` is I .+ I?

Moving on to proofs where we should rely on `. _` to have GHC help us out.

In [38]:
distl :: ((b .+ c) .* a) ~~ ((b .* a) .+ (c .* a))
distl = mulComm . distr . (lefting mulComm) . (righting mulComm)